In [7]:
import torch
inputs = torch.tensor([
    [
        0.43, 0.15, 0.89
    ],
    [0.55, 0.87, 0.66],
    [0.57, 0.85, 0.64],
    [0.22, 0.58, 0.33],
    [0.77, 0.25, 0.10],
    [0.05, 0.80, 0.55]

])
query = inputs[1]
attn_score2 = torch.empty(inputs.shape[0])
for i in range(len(inputs)):
    attn_score2[i] = torch.dot(inputs[i], query)
attn_weight2 = torch.softmax(attn_score2, dim=0)
print(attn_score2)
print(query.shape)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
torch.Size([3])


In [8]:
context_vec2 = torch.zeros(query.shape)
for i, xi in enumerate(inputs):
    context_vec2 += attn_weight2[i] * xi
print(context_vec2)

tensor([0.4419, 0.6515, 0.5683])


In [10]:
attn_scores = torch.empty(6, 6)
for i, xi in enumerate(inputs):
    for j, xii in enumerate(inputs):
        attn_scores[i, j] = torch.dot(xi, xii)
print(attn_scores)


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [15]:
# use @ for faster matrix multiplication
attn_scores = torch.empty(6, 6)
attn_scores = inputs @ inputs.T
attn_scores = torch.softmax(attn_scores, dim=-1)
all_context_vec = attn_scores @ inputs
print(all_context_vec)
print(attn_scores)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [ ]:
# create query, key, value matrices
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
torch.manual_seed(123)
w_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
w_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
w_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)

query_2 = x_2 @ w_query
key_2 = x_2 @ w_key
value_2 = x_2 @ w_value
print(query_2)

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)


In [ ]:
#Get query, key, value matrices
#6*3  @ 3*2 = 6*2
query = inputs @ w_query
key = inputs @ w_key
value = inputs @ w_value
print(query, query.shape)
print(key, key.shape)

tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]], grad_fn=<MmBackward0>) torch.Size([6, 2])
tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>) torch.Size([6, 2])


In [ ]:
# calculate attention score 
keys_2 = key[1]
atten_score2 = query_2.dot(keys_2)
atten_score2 = query_2 @ key.T
print(atten_score2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)


In [28]:
# convert attention score to attention weight
d_k = key.shape[-1]
attn_weight2 = torch.softmax(atten_score2 / d_k**0.5, dim=-1)
print(attn_weight2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)


In [29]:
#calculate context vector
context_vec2 = attn_weight2 @ value
print(context_vec2)

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)


In [36]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.w_query = nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
        self.w_key = nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
        self.w_value = nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
    
    def forward(self, x):
        keys = x @ self.w_key
        query = x @ self.w_query
        value = x @ self.w_value
        atten_score = query @ keys.T
        d_k = keys.shape[-1]
        atten_weights = torch.softmax(atten_score / d_k**0.5, dim=-1)
        contex_vec = atten_weights @ value
        return contex_vec
        

In [37]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1.forward(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [39]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
    def forward(self, x):
        query = self.w_query(x)
        key = self.w_key(x)
        value = self.w_value(x)
        atten_weights = torch.softmax(query @ key.T / key.shape[-1]**0.5, dim=-1)
        context_vec = atten_weights @ value
        return context_vec

In [ ]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2.forward(inputs)) # context vec
queries = sa_v2.w_query(inputs)
keys = sa_v2.w_key(inputs)
atten_score = queries @ keys.T
atten_weight = torch.softmax(atten_score / keys.shape[-1]**0.5, dim=-1)
print(atten_weight)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [46]:
context_length = atten_score.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
masked_simpe = atten_weight * mask_simple
print(masked_simpe)

row_sums = masked_simpe.sum(dim=-1, keepdim=True)
masked_simpe_norm = masked_simpe / row_sums
print(masked_simpe_norm)


tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [48]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
print(dropout(atten_weight))

tensor([[0.3843, 0.3293, 0.0000, 0.3100, 0.3442, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.2992, 0.0000, 0.2955],
        [0.4071, 0.3318, 0.3325, 0.2996, 0.0000, 0.2961],
        [0.0000, 0.3334, 0.3337, 0.0000, 0.0000, 0.3128],
        [0.0000, 0.3337, 0.0000, 0.3177, 0.0000, 0.3169],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


In [49]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [50]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, bias=False):
        super().__init__()
        self.w_query = torch.nn.Linear(d_in, d_out, bias=bias)
        self.w_key = torch.nn.Linear(d_in, d_out, bias=bias)
        self.w_value = torch.nn.Linear(d_in, d_out, bias=bias)
        self.d_out = d_out
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer(
            'mask', 
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.w_key(x)
        queries = self.w_query(x)
        values = self.w_value(x)
        
        atten_score = queries @ keys.transpose(1, 2)
        atten_score.masked_fill(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )
        atten_weight = torch.softmax(atten_score / keys.shape[-1]**0.5, dim=-1)
        atten_weight = self.dropout(atten_weight)
        
        context_vec = atten_weight @ values
        return context_vec
        

In [51]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length=context_length, dropout=0.0)
context_vec = ca(batch)
print(context_vec.shape)

torch.Size([2, 6, 2])


In [53]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(
                d_in=d_in, d_out=d_out, context_length=context_length, dropout=dropout, bias=bias
            ) for _ in range(num_heads)]
        )
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1) # head(x) = head.forward(x)

In [55]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in=d_in, d_out=d_out, context_length=context_length, dropout=0.0, num_heads=2
)
context_vec = mha(batch)
print(context_vec)
print(context_vec.shape)

tensor([[[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 4])


In [56]:
class MiltiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, bias, dropout, num_heads):
        super().__init__()
        assert (d_out % num_heads==0), \
            "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out
        self.w_query = torch.nn.Linear(d_in, d_out, bias=bias)
        self.w_value = torch.nn.Linear(d_in, d_out, bias=bias)
        self.w_key = torch.nn.Linear(d_in, d_out, bias=bias)
        self.out_proj = torch.nn.Linear(d_in, d_out)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer(
            "mask", 
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        
    def forward(self, x):
        b, num_tokens, d_out = x.shape
        keys = self.w_key(x)
        queries = self.w_query(x)
        values = self.w_value(x)
        
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        
        atten_score = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        atten_score.masked_fill_(mask_bool, -torch.inf)
        atten_weight = torch.softmax(atten_score / keys.shape[-1]**0.5, dim=-1)
        atten_weight = self.dropout(atten_weight)
        context_vec = (atten_weight @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        context_vec = self.out_proj(context_vec)
        return context_vec
        
        